# Graph runbook 2

In [ ]:
%%bash
docker run --name age -d -e POSTGRES_HOST_AUTH_METHOD=trust -p 5432:5432 apache/age:release_PG16_1.5.0
sleep 10

In [ ]:
%%bash
psql -h localhost -U postgres -d postgres -c "

CREATE EXTENSION IF NOT EXISTS age;

"

In [ ]:
%%bash
psql -h localhost -U postgres -d postgres -c "

LOAD 'age';
SET search_path = ag_catalog, postgres, public;

SELECT create_graph('social_network');

"

In [ ]:
%%bash
psql -h localhost -U postgres -d postgres -c "

LOAD 'age';
SET search_path = ag_catalog, postgres, public;

SELECT * 
FROM cypher('social_network', \$\$
    CREATE 
        (vemund:Human   {name:'Vemund Santi', age:29}),
        (kristian:Human {name:'Kristian Vikenes', age:27}),
        (trine:Human    {name:'Trine Hansen', age:31}),
        (barten:Cat     {name:'Barten', age:7}),
        (yava:Dog       {name:'Yava', age:5}),

        (vemund)-[:friend_with]->(kristian),
        (vemund)-[:friend_with]->(trine),

        (trine)-[:owner_of]->(barten),
        (kristian)-[:owner_of]->(yava)
\$\$) as (v agtype);

"

In [ ]:
import json
import psycopg
from psycopg.abc import Query


def _(vertex: str):
    return json.loads(vertex[:-8])['properties']['name']


def __(edge: str):
    return json.loads(edge[:-6])['label']


def query(sql: Query) -> None:
    with psycopg.connect("host=localhost dbname=postgres user=postgres") as conn:
        with conn.cursor() as cur:
            cur.execute("LOAD 'age';")
            cur.execute('SET search_path = ag_catalog, postgres, public;')

            cur.execute(sql)

            for res in cur.fetchall():
                print(f"{_(res[0])} -[{__(res[1])}]-> {_(res[2])}")

In [ ]:
# Get everything that has a relation
query("""
    SELECT * 
    FROM cypher('social_network', $$
        MATCH (start)-[rel]->(dest)
        RETURN start, rel, dest
    $$) as (start agtype, rel agtype, dest agtype);
""")

In [ ]:
# Get all of Vemund's friends
query("""
    SELECT * 
    FROM cypher('social_network', $$
        MATCH (vemund:Human {name:'Vemund Santi'})-[rel:friend_with]->(friend)
        RETURN vemund, rel, friend
    $$) as (vemund agtype, rel agtype, friend agtype);
""")

In [ ]:
# Get all of Vemund's friends that own a dog
query("""
    SELECT *
    FROM cypher('social_network', $$
        MATCH (start1:Human)-[rel1:friend_with]->(dest1)-[rel2:owner_of]->(dest2:Dog)
        RETURN start1, rel1, dest1
    $$) as (start agtype, rel agtype, dest agtype);
""")

In [ ]:
%%bash
psql -h localhost -U postgres -d postgres -c "

CREATE TABLE IF NOT EXISTS employee (
    name TEXT PRIMARY KEY,
    company TEXT NOT NULL
);
"

In [ ]:
%%bash
psql -h localhost -U postgres -d postgres -c "

INSERT INTO employee (name, company)
VALUES ('Kristian Vikenes', 'Oiiku'),
       ('Trine Hansen', 'Age Labs');
"

In [ ]:
%%bash
psql -h localhost -U postgres -d postgres -c "

LOAD 'age';
SET search_path = ag_catalog, postgres, public;

-- Get all of Vemund's friends that own a dog and join them with employee information
SELECT *
FROM employee
JOIN cypher('social_network', \$\$
        MATCH (start1:Human)-[rel1:friend_with]->(dest1)-[rel2:owner_of]->(dest2:Dog)
        RETURN start1, rel1, dest1.name
    \$\$) as (start agtype, rel agtype, dest TEXT)
 ON employee.name = dest;
"

In [ ]:
%%bash
docker stop age
docker rm age